In [1]:
### 本节主要实现使用pytorch来完成线性回归

In [2]:
import torch
import numpy as np

In [3]:
## 生成数据集

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [4]:
### 读取数据

In [6]:
import torch.utils.data as Data

batch_size = 10
dataset = Data.TensorDataset(features, labels)      # 组合训练数据的特征和标签信息
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)  # 随机读取小批量

for X, y in data_iter:
    print(X, y)
    break

tensor([[ 1.4853,  1.4610],
        [-0.4883, -1.4545],
        [-0.6530,  0.8707],
        [-0.6345, -1.0553],
        [ 0.3513,  1.0692],
        [ 0.7050,  0.3915],
        [-0.1605,  0.5744],
        [-1.3243, -0.2287],
        [ 0.4729,  1.2639],
        [ 0.5773, -0.8693]]) tensor([ 2.1918,  8.1612, -0.0733,  6.5110,  1.2669,  4.2738,  1.9286,  2.3280,
         0.8369,  8.3123])


In [7]:
### 定义模型

In [11]:
import torch.nn as nn

class LinearRegNet(nn.Module):   # 新定义的模型类继承自nn.Module
    def __init__(self, n_feature):
        super(LinearRegNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)  # 先定义每个层是什么
        
    def forward(self, x): ## 定义前向传播
        y = self.linear(x)          # 调用定义好的层，指明每层的输入数据
        return y
    
net = LinearRegNet(num_inputs) # 初始化，输入特征信息（信息里面包含了维度信息）
print(net)  ## print可以打印网络结构

LinearRegNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [12]:
# # 此处也可以使用 nn.Sequential() 来生成网络模型
# ## 写法一
# net = nn.Sequential(
#     nn.Linear(num_inputs, 1)
# )

# ## 写法二
# net = nn.Sequential()
# net.add_module('linear', nn.Linear(num_inputs, 1))
# # net.add_module()

# ## 写法三
# from collections import OrderedDict
# net = nn.Sequential(OrderedDict([
#             ('linear', nn.Linear(num_inputs, 1))
#             ### ()
#         ]))

# print(net)
# print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [13]:
# 将待学习参数打印出来
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.2583,  0.6369]], requires_grad=True)
Parameter containing:
tensor([-0.2126], requires_grad=True)


In [14]:
### 初始化模型参数

In [15]:
from torch.nn import init ## init是initializer的缩写

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0.01)

Parameter containing:
tensor([0.0100], requires_grad=True)

In [16]:
### 定义损失函数
loss = nn.MSELoss()

In [17]:
### 定义优化算法

In [19]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [20]:
# 如果想要改变学习率的值，可以对其调整

for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

In [ ]:
### 训练模型

In [24]:
num_epoch = 15
for epoch in range(1, num_epoch+1):
    for X, y in data_iter:
        out = net(X)           # 模型输出 
        l = loss(out, y.view(-1, 1))  # 求损失值 
        optimizer.zero_grad()  # 梯度置零 
        l.backward()           # 反向传播 
        optimizer.step()       # 在optimizer的指导下，更新参数 
        
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000066
epoch 2, loss: 0.000090
epoch 3, loss: 0.000129
epoch 4, loss: 0.000161
epoch 5, loss: 0.000033
epoch 6, loss: 0.000106
epoch 7, loss: 0.000065
epoch 8, loss: 0.000058
epoch 9, loss: 0.000078
epoch 10, loss: 0.000142
epoch 11, loss: 0.000177
epoch 12, loss: 0.000092
epoch 13, loss: 0.000109
epoch 14, loss: 0.000095
epoch 15, loss: 0.000093


In [25]:
# 获取模型参数
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9997, -3.4000]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1992], requires_grad=True)


In [ ]:
### 小结：

# 在数据准备阶段，torch.utils.data 提供了封装训练数据和标签的函数
# 在定义模型阶段，torch.nn模块提供了很多预定义好的层
# 在定义模型阶段，torch.nn.init可以为待学习参数进行多种方式的初始化
# 在优化算法中，  torch.optim定义了多种优化方法